### Test function development for the machine_learning.py module

In [9]:
from ramandecompy import machine_learning

In [10]:
ml.keyfinder('ramandecompy/tests/test_files/test_experiment.hdf5')

['300C/25s', '300C/35s', '300C/45s', '300C/55s', '300C/65s']

In [11]:
import h5py

In [12]:
hdf5 = h5py.File('ramandecompy/tests/test_files/test_experiment.hdf5', 'r')

In [13]:
type(hdf5['300C'])

h5py._hl.group.Group

In [14]:
type(hdf5['300C/25s'])

h5py._hl.group.Group

In [15]:
hdf5['300C/25s'].keys()

KeysView(<HDF5 group "/300C/25s" (21 members)>)

In [21]:
# def test_keyfinder(hdf5_filename):
"""
docstring
"""
hdf5_filename = 'ramandecompy/tests/test_files/test_experiment.hdf5'
key_list = machine_learning.keyfinder(hdf5_filename)
assert isinstance(key_list, list), 'expected output is not a list'
assert isinstance(key_list[0], str), 'first element of list should be a string'
assert len(key_list) == 5, 'number of keys in hdf5 file is incorrect'
try:
    machine_learning.keyfinder(4.2)
except TypeError:
    print('A float was passed to the function and was handled well with a TypeError')
try:
    machine_learning.keyfinder('hdf5_filename.txt')
except TypeError:
    print('A .txt was passed to the function and was handled well with a TypeError')


A float was passed to the function and was handled well with a TypeError
A .txt was passed to the function and was handled well with a TypeError


In [18]:
len(key_list)

5